In [7]:
#Here we install the package. For me it's been a nightmare to install rdkit into Kaggle's environment. 
#But wonderful Kaggle's technical support helped me to find the way.

!conda install -y -c rdkit rdkit;
!pip install pandas==0.23.0

Solving environment: \ 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/linux-64::scipy==1.1.0=py36hfc37229_0
  - defaults/linux-64::pycparser==2.18=py36hf9f622e_1
  - defaults/linux-64::bleach==2.1.3=py36_0
  - defaults/linux-64::ipywidgets==7.2.1=py36_0
  - defaults/linux-64::pyodbc==4.0.23=py36hf484d3e_0
  - defaults/linux-64::matplotlib==2.2.2=py36h0e671d2_1
  - defaults/linux-64::imagesize==1.0.0=py36_0
  - defaults/linux-64::jupyterlab_launcher==0.10.5=py36_0
  - defaults/linux-64::cython==0.28.2=py36h14c3975_0
  - defaults/linux-64::numba==0.38.0=py36h637b7d7_0
  - defaults/linux-64::sortedcollections==0.6.1=py36_0
  - defaults/linux-64::curl==7.60.0=h84994c4_0
  - defaults/linux-64::lazy-object-proxy==1.3.1=py36h10fcdad_0
  - defaults/linux-64::click==6.7=py36h5253387_0
  - defaults/linux-64::pytest==3.5.1=py36_0
  - defaults/linux-64::nose==1.3.7=py36hcdf7029_2
  - defaults/linux-64::a

In [8]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

As you see the only information we have here is SMILES representation of molecular formulas. But RDkit is able to work with MOL
representations. And it's actually nice to know RDkit still provides an opportunity to transform SMILES to MOL. Let's check what we can do.   

In [9]:
# AddHs function adds H atoms to a MOL (as Hs in SMILES are usualy ignored)
# GetNumAtoms() method returns a general nubmer of all atoms in a molecule
# GetNumHeavyAtoms() method returns a nubmer of all atoms in a molecule with molecular weight > 1


df['mol'] = df['mol'].apply(lambda x: Chem.AddHs(x))
df['num_of_atoms'] = df['mol'].apply(lambda x: x.GetNumAtoms())
df['num_of_heavy_atoms'] = df['mol'].apply(lambda x: x.GetNumHeavyAtoms())

NameError: name 'df' is not defined

In [10]:
#Train the model
ridge = RidgeCV(cv=5)
ridge.fit(X_train, y_train)
#Evaluate results
evaluation(ridge, X_test, y_test)

NameError: name 'RidgeCV' is not defined

In [11]:
from rdkit.Chem import Descriptors
df['tpsa'] = df['mol'].apply(lambda x: Descriptors.TPSA(x))
df['mol_w'] = df['mol'].apply(lambda x: Descriptors.ExactMolWt(x))
df['num_valence_electrons'] = df['mol'].apply(lambda x: Descriptors.NumValenceElectrons(x))
df['num_heteroatoms'] = df['mol'].apply(lambda x: Descriptors.NumHeteroatoms(x))

NameError: name 'df' is not defined

Let's check the improvement of the model perfomance with new features.

In [12]:
train_df = df.drop(columns=['smiles', 'mol', 'logP'])
y = df['logP'].values

print(train_df.columns)

#Perform a train-test split. We'll use 10% of the data to evaluate the model while training on 90%

X_train, X_test, y_train, y_test = train_test_split(train_df, y, test_size=.1, random_state=1)

NameError: name 'df' is not defined

In [ ]:
#Train the model
ridge = RidgeCV(cv=5)
ridge.fit(X_train, y_train)
#Evaluate results and plot predictions
evaluation(ridge, X_test, y_test)

# mol2vec - learning vector representations of molecular subctructures

From package description 'Mol2vec is an unsupervised machine learning approach to obtain high dimensional embeddings of chemical substructures. It is based on word2vec algorithm and if word2vec learns such embeddings that vectors representing similar words group together, so should Mol2vec learn substructure embeddings where vectors of chemically related substructures end up close in vector space'. 

In other words mol2vec takes inspiration in one of the most widespread NLP technique.
Without going deep in detaials we can describe the process as following: 

**1.** A molecule is divided into substructures of a fixed radius (usually encoding distance to other atoms in bonds, i.e. radius = 1 represents a single heavy atom or a group of closest atoms around a heavy atom), substructures are encoded via [Morgan fingerprints](https://docs.chemaxon.com/display/docs/Extended+Connectivity+Fingerprint+ECFP) with the help of RDkit. Substructures reflect words in 'chemical sentences'.

**2.** These encoded substructures are fed to Word2vec yielding vector representations of substructures in a number of dimensions according to their occurance together.

**3.** Summing up substructure vectors we get vector representations of whole molecules.


The main idea behind is that we can use vector representations of molecules in different dimensions as features in supervised learning, supposing similar molecules will have close vector representations and will have close target values. 

Another wonderful point is that mol2vec provides a pre-trained model which has learned 20M different compounds with radius=1 for Morgan fingerprints.

In [13]:
#Installing a package
!pip install git+https://github.com/samoturk/mol2vec;

  Cloning https://github.com/samoturk/mol2vec to /tmp/pip-req-build-io3nq5r9
  Running command git clone -q https://github.com/samoturk/mol2vec /tmp/pip-req-build-io3nq5r9
  Resolved https://github.com/samoturk/mol2vec to commit 850d944d5f48a58e26ed0264332b5741f72555aa
  Created wheel for mol2vec: filename=mol2vec-0.1-py3-none-any.whl size=14038 sha256=5fa60a00315374732c7cf8ab8bb1bca5fd702477dcbba4bdb0a9f2c664962a0e
  Stored in directory: /tmp/pip-ephem-wheel-cache-g1suexll/wheels/e1/f8/44/e7b5a66cec5932da508b308257d09bb5ac929b57e0641973f6
Successfully built mol2vec


In [20]:
import pandas as pd
from rdkit import Chem

In [17]:
data = pd.read_csv('/kaggle/input/clean-dta/fine_data.csv')

We'll start from scratch with initial dataset.

In [33]:
#Transforming SMILES to MOL
data['mol'] = data['SMILES'].apply(lambda x: Chem.MolFromSmiles(x))

RDKit WARNING: [00:43:26] Conflicting single bond directions around double bond at index 55.
RDKit WARNING: [00:43:26]   BondStereo set to STEREONONE and single bond directions set to NONE.


Once again we need to transform SMILES representations to MOL.

In [22]:
from rdkit import DataStructs
from rdkit.Chem import AllChem, Draw, Descriptors
from rdkit.Chem.Draw import IPythonConsole
from sklearn.preprocessing import FunctionTransformer

In [23]:
#Loading pre-trained model via word2vec
from gensim.models import word2vec
model = word2vec.Word2Vec.load('../input/mlchem/model_300dim.pkl')

In [34]:
from mol2vec.features import mol2alt_sentence, mol2sentence, MolSentence, DfVec, sentences2vec
from gensim.models import word2vec
print('Molecular sentence:', mol2alt_sentence(data['mol'][1], radius=1))
print('\nMolSentence object:', MolSentence(mol2alt_sentence(data['mol'][1], radius=1)))
print('\nDfVec object:',DfVec(sentences2vec(MolSentence(mol2alt_sentence(data['mol'][1], radius=1)), model, unseen='UNK')))

Molecular sentence: ['2246728737', '3542456614', '2245384272', '592593828', '848128881', '3010410747', '2245384272', '592593828', '2246728737', '3542456614', '2245384272', '603510687', '2245384272', '751208592', '847336149', '3188978453', '3217380708', '1217367676', '2976816164', '601754018', '2246728737', '1861965050', '2968968094', '1824088295', '2976033787', '2975316496', '2976816164', '3371487996', '2246728737', '1861965050', '2246728737', '1861965050', '2968968094', '2117068077', '2968968094', '2119439498']

MolSentence object: MolSentence with 36 words

DfVec object: (36, 300) dimensional vector


In [35]:
#Constructing sentences
data['sentence'] = data.apply(lambda x: MolSentence(mol2alt_sentence(x['mol'], 1)), axis=1)

#Extracting embeddings to a numpy.array
#Note that we always should mark unseen='UNK' in sentence2vec() so that model is taught how to handle unknown substructures
data['mol2vec'] = [DfVec(x) for x in sentences2vec(data['sentence'], model, unseen='UNK')]
X = np.array([x.vec for x in data['mol2vec']])
X.shape

(1257, 300)

In [36]:
# df = pd.read_csv('/kaggle/input/hackerton/1400.xlsx - Smile-IC50-CC50.csv')
data['mol'] = data['SMILES'].apply(lambda x: Chem.MolFromSmiles(x))

print('Molecular sentence:', mol2alt_sentence(data['mol'][1], radius=1))
print('\nMolSentence object:', MolSentence(mol2alt_sentence(data['mol'][1], radius=1)))
print('\nDfVec object:',DfVec(sentences2vec(MolSentence(mol2alt_sentence(data['mol'][1], radius=1)), model, unseen='UNK')))

RDKit WARNING: [00:43:44] Conflicting single bond directions around double bond at index 55.
RDKit WARNING: [00:43:44]   BondStereo set to STEREONONE and single bond directions set to NONE.


Molecular sentence: ['2246728737', '3542456614', '2245384272', '592593828', '848128881', '3010410747', '2245384272', '592593828', '2246728737', '3542456614', '2245384272', '603510687', '2245384272', '751208592', '847336149', '3188978453', '3217380708', '1217367676', '2976816164', '601754018', '2246728737', '1861965050', '2968968094', '1824088295', '2976033787', '2975316496', '2976816164', '3371487996', '2246728737', '1861965050', '2246728737', '1861965050', '2968968094', '2117068077', '2968968094', '2119439498']

MolSentence object: MolSentence with 36 words

DfVec object: (36, 300) dimensional vector


In [38]:
#Constructing sentences
data['sentence'] = data.apply(lambda x: MolSentence(mol2alt_sentence(x['mol'], 1)), axis=1)

# y1y2= data['IC50','CC50l']

#Extracting embeddings to a numpy.array
#Note that we always should mark unseen='UNK' in sentence2vec() so that model is taught how to handle unknown substructures
data['mol2vec'] = [DfVec(x) for x in sentences2vec(data['sentence'], model, unseen='UNK')]
X = np.array([x.vec for x in data['mol2vec']])
# y = CC50.values ""


In [39]:
X.shape

(1257, 300)

In [40]:
DF = pd.DataFrame(X)

In [41]:
DF.head()


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299
0,-0.210848,-1.585430,-0.118545,-2.438784,1.577855,-1.791306,-5.109906,3.512098,-0.191232,-1.455992,4.821575,1.178078,0.637878,1.274955,-1.080271,-2.178324,3.464671,-1.637504,-3.468479,3.393809,5.409534,0.194065,8.144125,3.729299,-7.401216,-1.703439,-4.818856,-4.640135,3.184485,-3.476864,3.152993,-2.439728,-3.028374,0.719387,0.201592,1.431040,-3.243760,-0.983962,9.060383,5.603230,...,2.416070,3.354124,1.229612,4.539350,1.913098,-2.460130,-0.630794,1.408774,4.814612,-3.310879,5.220927,5.923298,-0.983659,-2.753516,1.564373,-6.155665,-0.301778,1.037219,-2.179899,-0.278502,-3.610864,-1.098709,-4.796883,-2.114510,-0.639391,6.050268,0.421129,-0.246678,-0.849338,1.342512,0.395349,5.072766,1.145650,-0.474437,-4.497627,-5.413200,-5.711426,-2.469884,-5.438836,1.620259
1,-1.229343,-1.249687,-0.375884,-3.223105,3.549318,-1.426894,-6.608445,3.539260,0.982967,-2.582242,4.315680,2.754913,1.099961,2.073844,-3.176532,-3.830537,5.306746,-2.701547,-2.131070,6.040267,4.109839,1.146983,10.905228,6.087926,-6.212910,-3.465320,-4.925318,-5.307298,4.548211,-2.420787,1.906033,-2.538707,-3.580737,-0.864888,0.249586,0.569357,-3.481503,-0.419450,9.079836,6.863696,...,0.840615,4.177062,2.373032,5.502800,2.644839,-3.562375,-2.765844,1.884635,6.275490,-3.965557,4.034759,5.892056,-1.998246,-1.295757,1.259240,-7.385296,-0.023575,0.634289,-3.180566,0.875724,-3.983332,-1.883367,-6.671087,-3.453202,0.502125,5.850771,0.391654,1.465993,-0.515839,0.986472,1.663535,5.261796,4.314438,-0.115328,-5.096564,-5.632138,-6.515070,-3.512572,-6.707908,-0.930133
2,-0.335091,-1.052644,0.148067,-3.004084,2.315342,-1.410794,-4.672865,4.509478,0.801336,-1.973814,4.716659,1.001105,0.711065,1.120683,-1.563200,-3.400352,4.831672,-2.535574,-2.391935,3.785268,5.400168,0.541877,9.679538,3.992936,-5.691163,-2.090305,-5.165671,-4.659379,4.773184,-2.335523,2.134538,-2.597796,-3.841131,-0.918188,1.102047,0.936889,-3.165414,-0.537165,9.680660,6.586655,...,1.922011,4.253828,1.625621,4.817252,2.549628,-3.181868,-0.389623,2.526728,5.801613,-3.588546,5.817666,6.715818,-1.049073,-1.952821,2.391051,-7.040460,0.045488,0.694680,-2.057571,0.124702,-4.357765,-2.391603,-4.583930,-1.670775,1.373234,6.645916,1.001134,-0.236938,-1.030301,-0.063074,1.250266,5.929393,2.094081,0.293433,-4.677534,-5.573073,-6.414720,-3.094424,-7.023509,0.889786
3,-1.237990,-2.146065,-2.041042,-3.261227,0.628436,-3.924288,-6.218499,3.734038,1.658088,-0.739417,3.603770,1.076025,2.363109,2.997272,-2.268878,-3.441488,6.009953,-2.921525,-2.033161,2.614233,5.145013,1.055756,11.180459,3.310329,-3.754476,-0.673463,-3.900088,-3.622491,3.972930,-3.394971,-0.449827,-3.328115,-4.605798,0.325274,-0.825034,0.827432,-1.663674,-0.835978,9.780725,6.783730,...,3.014049,5.201584,2.950154,7.435416,3.076124,-2.678558,-0.446916,1.867724,4.381358,-2.860447,5.562176,6.576489,1.563389,-2.171316,2.886764,-5.325635,-1.987993,-0.094358,-2.728430,0.535264,-6.098519,-3.631428,-6.574777,-1.046956,1.386063,8.072800,0.131481,-0.582024,-1.764034,-0.240916,0.559931,8.525725,2.371636,1.024816,-6.790434,-6.704412,-6.563089,-4.627900,-8.825220,0.141596
4,2.471257,-3.521509,-0.947530,-1.953958,3.825321,0.132207,-11.256601,5.903816,0.307614,-1.004157,4.434114,2.435546,1.836821,4.638535,0.566277,0.279551,7.099013,-2.526517,-3.003318,2.987564,6.650794,-0.322288,13.076282,7.464808,-6.663782,-4.862357,-4.212073,-6.686661,8.222354,-6.553490,1.957072,1.495035,-2.658622,-0.151429,-0.247888,3.040978,-4.038055,0.851044,10.136612,7.085483,...,1.177417,6.544284,4.458224,7.107502,1.781557,-8.030346,0.277483,2.226524,5.227225,0.098547,4.713666,5.216825,-0.316284,-1.689678,2.117877,-8.168522,0.943471,1.904706,-2.664252,3.059083,-7.108460,-1.917797,-7.852603,-0.796282,1.106541,8.563437,0.080413,2.269230,0.44000

In [43]:
DF['IC50'] = data['IC50']

In [44]:
DF['CC50'] = data['CC50']

And we got the best among our results. This one is actually a rather good result even for computional chemistry.

In [45]:
DF.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,IC50,CC50
0,-0.210848,-1.585430,-0.118545,-2.438784,1.577855,-1.791306,-5.109906,3.512098,-0.191232,-1.455992,4.821575,1.178078,0.637878,1.274955,-1.080271,-2.178324,3.464671,-1.637504,-3.468479,3.393809,5.409534,0.194065,8.144125,3.729299,-7.401216,-1.703439,-4.818856,-4.640135,3.184485,-3.476864,3.152993,-2.439728,-3.028374,0.719387,0.201592,1.431040,-3.243760,-0.983962,9.060383,5.603230,...,1.229612,4.539350,1.913098,-2.460130,-0.630794,1.408774,4.814612,-3.310879,5.220927,5.923298,-0.983659,-2.753516,1.564373,-6.155665,-0.301778,1.037219,-2.179899,-0.278502,-3.610864,-1.098709,-4.796883,-2.114510,-0.639391,6.050268,0.421129,-0.246678,-0.849338,1.342512,0.395349,5.072766,1.145650,-0.474437,-4.497627,-5.413200,-5.711426,-2.469884,-5.438836,1.620259,2.7,500.0
1,-1.229343,-1.249687,-0.375884,-3.223105,3.549318,-1.426894,-6.608445,3.539260,0.982967,-2.582242,4.315680,2.754913,1.099961,2.073844,-3.176532,-3.830537,5.306746,-2.701547,-2.131070,6.040267,4.109839,1.146983,10.905228,6.087926,-6.212910,-3.465320,-4.925318,-5.307298,4.548211,-2.420787,1.906033,-2.538707,-3.580737,-0.864888,0.249586,0.569357,-3.481503,-0.419450,9.079836,6.863696,...,2.373032,5.502800,2.644839,-3.562375,-2.765844,1.884635,6.275490,-3.965557,4.034759,5.892056,-1.998246,-1.295757,1.259240,-7.385296,-0.023575,0.634289,-3.180566,0.875724,-3.983332,-1.883367,-6.671087,-3.453202,0.502125,5.850771,0.391654,1.465993,-0.515839,0.986472,1.663535,5.261796,4.314438,-0.115328,-5.096564,-5.632138,-6.515070,-3.512572,-6.707908,-0.930133,9.9,144.0
2,-0.335091,-1.052644,0.148067,-3.004084,2.315342,-1.410794,-4.672865,4.509478,0.801336,-1.973814,4.716659,1.001105,0.711065,1.120683,-1.563200,-3.400352,4.831672,-2.535574,-2.391935,3.785268,5.400168,0.541877,9.679538,3.992936,-5.691163,-2.090305,-5.165671,-4.659379,4.773184,-2.335523,2.134538,-2.597796,-3.841131,-0.918188,1.102047,0.936889,-3.165414,-0.537165,9.680660,6.586655,...,1.625621,4.817252,2.549628,-3.181868,-0.389623,2.526728,5.801613,-3.588546,5.817666,6.715818,-1.049073,-1.952821,2.391051,-7.040460,0.045488,0.694680,-2.057571,0.124702,-4.357765,-2.391603,-4.583930,-1.670775,1.373234,6.645916,1.001134,-0.236938,-1.030301,-0.063074,1.250266,5.929393,2.094081,0.293433,-4.677534,-5.573073,-6.414720,-3.094424,-7.023509,0.889786,8.3,500.0
3,-1.237990,-2.146065,-2.041042,-3.261227,0.628436,-3.924288,-6.218499,3.734038,1.658088,-0.739417,3.603770,1.076025,2.363109,2.997272,-2.268878,-3.441488,6.009953,-2.921525,-2.033161,2.614233,5.145013,1.055756,11.180459,3.310329,-3.754476,-0.673463,-3.900088,-3.622491,3.972930,-3.394971,-0.449827,-3.328115,-4.605798,0.325274,-0.825034,0.827432,-1.663674,-0.835978,9.780725,6.783730,...,2.950154,7.435416,3.076124,-2.678558,-0.446916,1.867724,4.381358,-2.860447,5.562176,6.576489,1.563389,-2.171316,2.886764,-5.325635,-1.987993,-0.094358,-2.728430,0.535264,-6.098519,-3.631428,-6.574777,-1.046956,1.386063,8.072800,0.131481,-0.582024,-1.764034,-0.240916,0.559931,8.525725,2.371636,1.024816,-6.790434,-6.704412,-6.563089,-4.627900,-8.825220,0.141596,39.4,143.0
4,2.471257,-3.521509,-0.947530,-1.953958,3.825321,0.132207,-11.256601,5.903816,0.307614,-1.004157,4.434114,2.435546,1.836821,4.638535,0.566277,0.279551,7.099013,-2.526517,-3.003318,2.987564,6.650794,-0.322288,13.076282,7.464808,-6.663782,-4.862357,-4.212073,-6.686661,8.222354,-6.553490,1.957072,1.495035,-2.658622,-0.151429,-0.247888,3.040978,-4.038055,0.851044,10.136612,7.085483,...,4.458224,7.107502,1.781557,-8.030346,0.277483,2.226524,5.227225,0.098547,4.713666,5.216825,-0.316284,-1.689678,2.117877,-8.168522,0.943471,1.904706,-2.664252,3.059083,-7.108460,-1.917797,-7.852603,-0.796282,1.106541,8.563437,0.080413,2.269230,0.440006,2.413494,-0.239113,2.899925,7.020583,-0.01524

In [46]:
DF.to_csv('/kaggle/working/cleanest.csv')